In [1]:
import os
import argparse
import numpy as np
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau, LambdaLR
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from network.model_unet_a_2d import *
from loss_utils import *
from data_loader import *
from data_augmentation import *
from test_utils import model_predict, dataset_eval
from torchinfo import summary
import random
import torch.backends.cudnn as cudnn
from torch.utils.data.sampler import Sampler
import itertools

In [2]:
def iterate_once(iterable):
    return np.random.permutation(iterable)


def iterate_eternally(indices):
    def infinite_shuffles():
        while True:
            yield np.random.permutation(indices)
    return itertools.chain.from_iterable(infinite_shuffles())


def grouper(iterable, n):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3) --> ABC DEF"
    args = [iter(iterable)] * n
    return zip(*args)


class TwoStreamBatchSampler(Sampler):
    """Iterate two sets of indices

    An 'epoch' is one iteration through the primary indices.
    During the epoch, the secondary indices are iterated through
    as many times as needed.
    """
    def __init__(self, primary_indices, secondary_indices, batch_size, secondary_batch_size):
        self.primary_indices = primary_indices
        self.secondary_indices = secondary_indices
        self.secondary_batch_size = secondary_batch_size
        self.primary_batch_size = batch_size - secondary_batch_size

        assert len(self.primary_indices) >= self.primary_batch_size > 0
        assert len(self.secondary_indices) >= self.secondary_batch_size > 0

    def __iter__(self):
        primary_iter = iterate_once(self.primary_indices)
        secondary_iter = iterate_eternally(self.secondary_indices)
        return (
            primary_batch + secondary_batch
            for (primary_batch, secondary_batch)
            in zip(grouper(primary_iter, self.primary_batch_size),
                    grouper(secondary_iter, self.secondary_batch_size))
        )

    def __len__(self):
        return len(self.primary_indices) // self.primary_batch_size


def generate_mask(img):
    """
    随机生成遮罩，返回遮罩及对应的损失遮罩。
    针对 H=1 的数据，确保 patch_x 至少为1，并处理随机起始位置的边界条件。
    """
    batch_size, channel, img_x, img_y = img.shape
    loss_mask = torch.ones(batch_size, img_x, img_y).cuda()
    mask = torch.ones(img_x, img_y).cuda()
    # 如果 img_x == 1，则 force patch_x 为1；否则按比例计算
    patch_x = int(img_x * 2 / 3) if img_x > 1 else 1
    patch_y = int(img_y * 2 / 3)
    # 计算随机起始位置时，防止负值
    if img_x - patch_x <= 0:
        w = 0
    else:
        w = np.random.randint(0, img_x - patch_x)
    if img_y - patch_y <= 0:
        h = 0
    else:
        h = np.random.randint(0, img_y - patch_y)
    mask[w:w + patch_x, h:h + patch_y] = 0
    loss_mask[:, w:w + patch_x, h:h + patch_y] = 0
    return mask.long(), loss_mask.long()


class DiceLoss(nn.Module):
    def __init__(self, n_classes):
        super(DiceLoss, self).__init__()
        self.n_classes = n_classes

    def _one_hot_encoder(self, input_tensor):
        tensor_list = []
        for i in range(self.n_classes):
            temp_prob = input_tensor == i * torch.ones_like(input_tensor)
            tensor_list.append(temp_prob)
        output_tensor = torch.cat(tensor_list, dim=1)
        return output_tensor.float()

    def _dice_loss(self, score, target):
        target = target.float()
        smooth = 1e-10
        intersect = torch.sum(score * target)
        y_sum = torch.sum(target * target)
        z_sum = torch.sum(score * score)
        loss = (2 * intersect + smooth) / (z_sum + y_sum + smooth)
        loss = 1 - loss
        return loss
    
    def _dice_mask_loss(self, score, target, mask):
        target = target.float()
        mask = mask.float()
        smooth = 1e-10
        intersect = torch.sum(score * target * mask)
        y_sum = torch.sum(target * target * mask)
        z_sum = torch.sum(score * score * mask)
        loss = (2 * intersect + smooth ) / (z_sum + y_sum + smooth)
        loss = 1 - loss
        return loss

    def forward(self, inputs, target, mask=None, weight=None, softmax=False, one_hot=False):
        # If softmax is True, apply softmax to the inputs
        if softmax:
            inputs = torch.softmax(inputs, dim=1)

        # One-hot encoding of target, only if one_hot is True
        if one_hot:
            target = self._one_hot_encoder(target)
        
        if weight is None:
            weight = [1] * self.n_classes

        assert inputs.size() == target.size(), 'predict & target shape do not match'
        
        class_wise_dice = []
        loss = 0.0
        if mask is not None:
            # Expand mask to match the number of classes
            mask = mask.repeat(1, self.n_classes, 1, 1).type(torch.float32)
            for i in range(0, self.n_classes): 
                dice = self._dice_mask_loss(inputs[:, i], target[:, i], mask[:, i])
                class_wise_dice.append(1.0 - dice.item())
                loss += dice * weight[i]
        else:
            for i in range(0, self.n_classes):
                dice = self._dice_loss(inputs[:, i], target[:, i])
                class_wise_dice.append(1.0 - dice.item())
                loss += dice * weight[i]
        
        return loss / self.n_classes
    

def mix_loss(output, img_l, patch_l, mask, l_weight=1.0, u_weight=0.5, unlab=False):
    CE = nn.CrossEntropyLoss(reduction='none')
    dice_loss = DiceLoss(n_classes=4)
    img_l, patch_l = img_l.type(torch.int64), patch_l.type(torch.int64)
    output_soft = F.softmax(output, dim=1)
    image_weight, patch_weight = l_weight, u_weight
    if unlab:
        image_weight, patch_weight = u_weight, l_weight
    patch_mask = 1 - mask
    loss_dice = dice_loss(output_soft, img_l, mask.unsqueeze(1)) * image_weight
    loss_dice += dice_loss(output_soft, patch_l, patch_mask.unsqueeze(1)) * patch_weight
    # loss_ce = image_weight * (CE(output, img_l) * mask).sum() / (mask.sum() + 1e-16) 
    # loss_ce += patch_weight * (CE(output, patch_l) * patch_mask).sum() / (patch_mask.sum() + 1e-16)#loss = loss_ce
    loss_ce = image_weight * (CE(output, img_l.argmax(dim=1)) * mask).sum() / (mask.sum() + 1e-16) 
    loss_ce += patch_weight * (CE(output, patch_l.argmax(dim=1)) * patch_mask).sum() / (patch_mask.sum() + 1e-16)#loss = loss_ce
    return loss_dice, loss_ce


def pre_train(model, snapshot_path, db_train_l, db_train_u, db_val):
    base_lr = 0.01
    num_classes = 4
    # max_iterations = 10000
    max_iterations = 1000
    batch_size = 24
    labeled_bs = 12
    seed = 42
    labeled_sub_bs, unlabeled_sub_bs = int(labeled_bs/2), int((batch_size-labeled_bs) / 2)

    def worker_init_fn(worker_id):
        random.seed(seed + worker_id)

    db_train = torch.utils.data.ConcatDataset([db_train_l, db_train_u])
    num_total = len(db_train)
    num_labeled = len(db_train_l)
    labeled_idxs = list(range(0, num_labeled))
    unlabeled_idxs = list(range(num_labeled, num_total))
    batch_sampler = TwoStreamBatchSampler(labeled_idxs, unlabeled_idxs, batch_size, batch_size-labeled_bs)

    trainloader = DataLoader(db_train, batch_sampler=batch_sampler, num_workers=4, pin_memory=True, worker_init_fn=worker_init_fn)

    valloader = DataLoader(db_val, batch_size=1, shuffle=False, num_workers=1)

    optimizer = torch.optim.SGD(model.parameters(), lr=base_lr, momentum=0.9, weight_decay=0.0001)

    model.train()
    iter_num = 0
    max_epoch = max_iterations // len(trainloader) + 1
    best_val_loss = np.inf
    iterator = tqdm(range(max_epoch), ncols=70)
    for _ in iterator:
        for _, (volume_batch, label_batch) in enumerate(trainloader):
            volume_batch, label_batch = volume_batch.cuda(), label_batch.cuda()

            img_a, img_b = volume_batch[:labeled_sub_bs], volume_batch[labeled_sub_bs:labeled_bs]
            lab_a, lab_b = label_batch[:labeled_sub_bs], label_batch[labeled_sub_bs:labeled_bs]
            img_mask, loss_mask = generate_mask(img_a)
            gt_mixl = lab_a * img_mask + lab_b * (1 - img_mask)

            #-- original
            net_input = img_a * img_mask + img_b * (1 - img_mask)
            out_mixl = model(net_input)
            loss_dice, loss_ce = mix_loss(out_mixl, lab_a, lab_b, loss_mask, u_weight=1.0, unlab=True)

            loss = (loss_dice + loss_ce) / 2            

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            iter_num += 1

            if iter_num > 0 and iter_num % 200 == 0:
                model.eval()
                val_loss = 0
                for _, sampled_batch in enumerate(valloader):
                    val_volume, val_label = sampled_batch
                    val_volume, val_label = val_volume.cuda(), val_label.cuda()
                    val_output = model(val_volume)
                    val_loss += DiceLoss(n_classes=4)(val_output, val_label).item()
                val_loss /= len(valloader)
                iterator.set_postfix({'Iter': iter_num, 'Val Loss': f'{val_loss:.4f}'})

                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    torch.save(model.state_dict(), snapshot_path)

                model.train()

            if iter_num >= max_iterations:
                break
        if iter_num >= max_iterations:
            iterator.close()
            break


def get_ACDC_masks(output):
    # probs = F.softmax(output, dim=1)
    _, probs = torch.max(output, dim=1)
    probs = F.one_hot(probs.squeeze(1).squeeze(1), num_classes=4).permute(0, 2, 1).unsqueeze(2).float()
    # if nms == 1:
    #     probs = get_ACDC_2DLargestCC(probs)      
    return probs

def sigmoid_rampup(current, rampup_length):
    """Exponential rampup from https://arxiv.org/abs/1610.02242"""
    if rampup_length == 0:
        return 1.0
    else:               
        current = np.clip(current, 0.0, rampup_length)
        phase = 1.0 - current / rampup_length
        return float(np.exp(-5.0 * phase * phase))
    
def get_current_consistency_weight(epoch):
    # Consistency ramp-up from https://arxiv.org/abs/1610.02242
    consistency = 0.1
    consistency_rampup = 200.0
    return 5* consistency * sigmoid_rampup(epoch, consistency_rampup)

def update_model_ema(model, ema_model, alpha):
    model_state = model.state_dict()
    model_ema_state = ema_model.state_dict()
    new_dict = {}
    for key in model_state:
        new_dict[key] = alpha * model_ema_state[key] + (1 - alpha) * model_state[key]
    ema_model.load_state_dict(new_dict)


def self_train(model, ema_model, snapshot_path, final_path, db_train_l, db_train_u, db_val):
    base_lr = 0.01
    num_classes = 4
    # max_iterations = 30000
    max_iterations = 1000
    batch_size = 24
    labeled_bs = 12
    seed = 42
    u_weight = 0.5
    labeled_sub_bs, unlabeled_sub_bs = int(labeled_bs/2), int((batch_size-labeled_bs) / 2)

    def worker_init_fn(worker_id):
        random.seed(seed + worker_id)

    db_train = torch.utils.data.ConcatDataset([db_train_l, db_train_u])
    num_total = len(db_train)
    num_labeled = len(db_train_l)
    labeled_idxs = list(range(0, num_labeled))
    unlabeled_idxs = list(range(num_labeled, num_total))
    batch_sampler = TwoStreamBatchSampler(labeled_idxs, unlabeled_idxs, batch_size, batch_size-labeled_bs)

    trainloader = DataLoader(db_train, batch_sampler=batch_sampler, num_workers=4, pin_memory=True, worker_init_fn=worker_init_fn)

    valloader = DataLoader(db_val, batch_size=1, shuffle=False, num_workers=1)

    optimizer = torch.optim.SGD(model.parameters(), lr=base_lr, momentum=0.9, weight_decay=0.0001)

    model.load_state_dict(torch.load(snapshot_path))    
    ema_model.load_state_dict(torch.load(snapshot_path))

    model.train()
    ema_model.train()

    iter_num = 0
    max_epoch = max_iterations // len(trainloader) + 1
    best_val_loss = np.inf
    iterator = tqdm(range(max_epoch), ncols=70)
    for _ in iterator:
        for _, sampled_batch in enumerate(trainloader):
            volume_batch, label_batch = sampled_batch
            volume_batch, label_batch = volume_batch.cuda(), label_batch.cuda()

            img_a, img_b = volume_batch[:labeled_sub_bs], volume_batch[labeled_sub_bs:labeled_bs]
            uimg_a, uimg_b = volume_batch[labeled_bs:labeled_bs + unlabeled_sub_bs], volume_batch[labeled_bs + unlabeled_sub_bs:]
            ulab_a, ulab_b = label_batch[labeled_bs:labeled_bs + unlabeled_sub_bs], label_batch[labeled_bs + unlabeled_sub_bs:]
            lab_a, lab_b = label_batch[:labeled_sub_bs], label_batch[labeled_sub_bs:labeled_bs]
            with torch.no_grad():
                pre_a = ema_model(uimg_a)
                pre_b = ema_model(uimg_b)
                plab_a = get_ACDC_masks(pre_a)
                plab_b = get_ACDC_masks(pre_b)
                img_mask, loss_mask = generate_mask(img_a)
                unl_label = ulab_a * img_mask + lab_a * (1 - img_mask)
                l_label = lab_b * img_mask + ulab_b * (1 - img_mask)
            consistency_weight = get_current_consistency_weight(iter_num//150)

            net_input_unl = uimg_a * img_mask + img_a * (1 - img_mask)
            net_input_l = img_b * img_mask + uimg_b * (1 - img_mask)
            out_unl = model(net_input_unl)
            out_l = model(net_input_l)
            unl_dice, unl_ce = mix_loss(out_unl, plab_a, lab_a, loss_mask, u_weight=u_weight, unlab=True)
            l_dice, l_ce = mix_loss(out_l, lab_b, plab_b, loss_mask, u_weight=u_weight)


            loss_ce = unl_ce + l_ce 
            loss_dice = unl_dice + l_dice

            loss = (loss_dice + loss_ce) / 2            

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            iter_num += 1
            update_model_ema(model, ema_model, 0.99)

            if iter_num > 0 and iter_num % 200 == 0:
                model.eval()
                val_loss = 0
                for _, sampled_batch in enumerate(valloader):
                    val_volume, val_label = sampled_batch
                    val_volume, val_label = val_volume.cuda(), val_label.cuda()
                    val_output = model(val_volume)
                    val_loss += DiceLoss(n_classes=4)(val_output, val_label).item()
                val_loss /= len(valloader)
                # print('Iter %d, Val Loss: %.4f' % (iter_num, val_loss))
                iterator.set_postfix({'Iter': iter_num, 'Val Loss': f'{val_loss:.4f}'})

                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    torch.save(model.state_dict(), final_path)
                model.train()

            if iter_num >= max_iterations:
                break
        if iter_num >= max_iterations:
            iterator.close()
            break

In [ ]:
th_delineation = 150
gpu = 3
aug = 2
deep_supervision = 0
torch.cuda.set_device(gpu)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Parameters
model_save_path = f'./checkpoints/unet_a_ds{int(deep_supervision)}_aug_{aug}_BCP.cross'
metrics_save_path = f'./metrics/unet_a_ds{int(deep_supervision)}_aug_{aug}_BCP.cross'
records_list = [5,10,20,50,160]
val_ratio = 0.2
df_list = []

for num_labeled in records_list:
    ## Train
    print(f"Number of labeled data: {num_labeled}")
    print("Training Stage")
    for fold in range(5):
        # if os.path.exists(f"{model_save_path}.num_labeled_{num_labeled}_fold_{fold}.final.pth"):
        #     continue
        # Set random seed for reproducibility
        seed = 42
        cudnn.benchmark = False
        cudnn.deterministic = True
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)

        x_train_l, y_train_l, _, _, _ = raw_data_load_ludb(40, num_labeled, fold, crop=[1250, 3750])
        x_train_u, y_train_u, _, _, _ = raw_data_load_rdb(400, 1999, fold, crop=[1250, 3750])

        num_val =  np.round(x_train_l.shape[0] * val_ratio).astype(int)
        x_val, y_val = x_train_l[:num_val], y_train_l[:num_val]
        x_train_l, y_train_l = x_train_l[num_val:], y_train_l[num_val:]

        print(f"Fold {fold+1}/{5}: Train labeled: {x_train_l.shape[0]}, Train unlabeled: {x_train_u.shape[0]}, Val: {x_val.shape[0]}")

        if aug == 0:
            db_train_l = ECGDataset(x_train_l, y_train_l, transform=base_transforms())
            db_train_u = ECGDataset(x_train_u, y_train_u, transform=base_transforms())
            db_val = ECGDataset(x_val, y_val, transform=base_transforms())
        elif aug == 1:
            db_train_l = ECGDataset(x_train_l, y_train_l, transform=get_train_transforms())
            db_train_u = ECGDataset(x_train_u, y_train_u, transform=get_train_transforms())
            db_val = ECGDataset(x_val, y_val, transform=get_train_transforms())
        elif aug == 2:
            db_train_l = ECGDataset(x_train_l, y_train_l, transform=base_transforms())
            db_train_l_aug = ECGDataset(x_train_l, y_train_l, transform=get_train_transforms())
            db_train_l = torch.utils.data.ConcatDataset([db_train_l, db_train_l_aug])
            db_train_u = ECGDataset(x_train_u, y_train_u, transform=base_transforms())
            db_train_u_aug = ECGDataset(x_train_u, y_train_u, transform=get_train_transforms())
            db_train_u = torch.utils.data.ConcatDataset([db_train_u, db_train_u_aug])
            db_val = ECGDataset(x_val, y_val, transform=base_transforms())
            db_val_aug = ECGDataset(x_val, y_val, transform=get_train_transforms())
            db_val = torch.utils.data.ConcatDataset([db_val, db_val_aug])
        else:
            raise ValueError("Invalid aug value. Choose from 0, 1, 2")

        model = UNet1D_A(length=2500, base_channels=16, kernel_size=9, dropout='channels', droprate=.2, num_classes=2).to('cuda')
        ini_ds, ini_aug = int(deep_supervision), int(aug)
        model_load_path = f"./checkpoints/unet_a_ds{ini_ds}.num_labeled_{num_labeled}_aug_{ini_aug}.fold_{fold}.epoch_20.pth"
        model.load_state_dict(torch.load(model_load_path))
        
        snapshot_path = f"{model_save_path}.num_labeled_{num_labeled}_fold_{fold}.pre_train.pth"
        pre_train(model, snapshot_path, db_train_l, db_train_u, db_val)

        final_path = f"{model_save_path}.num_labeled_{num_labeled}_fold_{fold}.final.pth"
        self_train(model, model, snapshot_path, final_path, db_train_l, db_train_u, db_val)

    ## Test
    print("Test Stage")
    data = []
    label = []
    preds = []
    seg_metrics_macro = []
    deli_metrics_macro = []
    
    for fold in range(5):
        model = UNet1D_A(length=2500, base_channels=16, kernel_size=9, dropout='channels', droprate=.2, num_classes=2).to(device)
        model_load_path = f"{model_save_path}.num_labeled_{num_labeled}_fold_{fold}.final.pth"
        model.load_state_dict(torch.load(model_load_path))

        x_train, y_train, _, x_test, y_test = raw_data_load_ludb(40, 160, fold, crop=[0, 5000])
        test_dataset = ECGDataset(x_test, y_test, transform=base_transforms())

        pred = model_predict(model, model_load_path, test_dataset, device, multi_lead_correction=False)
        
        flag_ludb = np.load('./dataset/ludb/flag.npy')
        index_shuffled_5fold = np.load('./dataset/ludb/ludb_index_shuffled_5fold_250113.npy')
        index_shuffled = index_shuffled_5fold[:,fold]
        index_shuffled_lead = []
        for i in np.array(index_shuffled):
            index_shuffled_lead.extend([k for k in range(12*i,12*i+12,1)])
        num_test = 40
        flag_test= flag_ludb[index_shuffled_lead[0:num_test*12]]
        dataset = (x_test, y_test, np.zeros((x_test.shape[0],)), flag_test)
        _, _, seg_metrics, deli_metrics = dataset_eval(dataset, pred, th_delineation=th_delineation, verbose=0)

        data.append(x_test)
        label.append(y_test)
        preds.append(pred)
        seg_metrics_macro.append(seg_metrics)
        deli_metrics_macro.append(deli_metrics)

    data = np.concatenate(data, axis=0)
    label = np.concatenate(label, axis=0)
    preds = np.concatenate(preds, axis=0)

    def summarize_total_rows(dfs):
        """
        Extracts 'Total' rows from DataFrames, calculates summary statistics,
        and returns a new DataFrame.

        Args:
        dfs: A list of pandas DataFrames with identical structure.

        Returns:
            A pandas DataFrame containing the mean, std, max, and min
            of each column of the 'Total' rows from all input DataFrames.
        """
        total_rows = [df[df['type'] == 'Total'].iloc[0] for df in dfs]
        total_df = pd.DataFrame(total_rows)

        # Get original headers, and remove 'type'
        original_headers = total_df.columns.tolist()
        original_headers.remove('type')


        # Calculate summary stats for each column (excluding type)
        summary_data = {
            'mean': total_df[original_headers].mean().to_list(),
            'std': total_df[original_headers].std().to_list(),
            'max': total_df[original_headers].max().to_list(),
            'min': total_df[original_headers].min().to_list()
        }
        # Create the summary DataFrame
        summary_df = pd.DataFrame(summary_data, index = original_headers)
        return summary_df
    
    # Macro average metrics of 5 folds
    seg_metrics_macro = summarize_total_rows(seg_metrics_macro)
    deli_metrics_macro = summarize_total_rows(deli_metrics_macro) 
    filtered_df = deli_metrics_macro[deli_metrics_macro.index.str.contains('f1')]
    merged_df_macro = pd.concat([seg_metrics_macro, filtered_df], axis = 0)                

    # Micro average metrics of 5 folds
    dataset = (data, label, np.zeros((data.shape[0],)), np.zeros((data.shape[0],)))
    _, _, seg_metrics_micro, deli_metrics_micro = dataset_eval(dataset, preds, th_delineation=th_delineation, verbose=0)
    # Filter df2 to include rows where column name contain 'f1'
    filtered_df = deli_metrics_micro[['type'] + [col for col in deli_metrics_micro.columns if 'f1' in col]]
    merged_df_micro = pd.merge(seg_metrics_micro, filtered_df, on='type', how='outer')
    micro_row = merged_df_micro[merged_df_micro['type'] == 'Total'].iloc[0]
    # Remove 'type' and convert to series
    micro_row_values = micro_row.drop('type')
    merged_df = copy.deepcopy(merged_df_macro)
    merged_df['micro'] = micro_row_values
    
    df_list.append(merged_df)
    # Final results
    print(merged_df)


# Save results
# 拼接数据，将总标题作为列上方的“标题行”
concat_frames = []
for i, df in enumerate(df_list):
    # 插入标题行
    df_with_title = df.copy()
    df_with_title.columns = pd.MultiIndex.from_tuples([(str(records_list[i]), col) for col in df.columns])
    concat_frames.append(df_with_title)

# 按列拼接，并保留行标题
result = pd.concat(concat_frames, axis=1)

# 写入 Excel
result.to_excel(f"{metrics_save_path}.xlsx")

Number of labeled data: 5
Training Stage
Fold 1/5: Train labeled: 48, Train unlabeled: 23970, Val: 12


 98%|██▉| 124/126 [14:53<00:14,  7.21s/it, Iter=1000, Val Loss=0.0992]


Fold 2/5: Train labeled: 48, Train unlabeled: 23970, Val: 12


 98%|██▉| 124/126 [14:56<00:14,  7.23s/it, Iter=1000, Val Loss=0.1420]


Fold 3/5: Train labeled: 48, Train unlabeled: 23970, Val: 12


...

 75%|████▌ | 3/4 [14:45<04:55, 295.04s/it, Iter=1000, Val Loss=0.0856]


Fold 5/5: Train labeled: 1536, Train unlabeled: 23970, Val: 384


 75%|████▌ | 3/4 [14:44<04:54, 294.92s/it, Iter=1000, Val Loss=0.1028]


Test Stage
                mean       std       max       min     micro
iou_p       0.813831  0.016827  0.828171  0.785289  0.808216
iou_qrs     0.892268  0.008258  0.905985  0.884357  0.890936
iou_t       0.854198  0.017577  0.881488  0.832701  0.854311
miou        0.853433  0.011239  0.871881  0.842286  0.851154
acc         0.934034  0.006154  0.944245  0.927565  0.933191
ave_f1      0.975334  0.003868  0.981258  0.971756  0.974246
f1_p_on     0.957892  0.005840  0.963778  0.949122  0.954387
f1_p_end    0.958137  0.005531  0.963484  0.949727  0.954617
f1_qrs_on   0.991753  0.002680  0.994720  0.988444  0.991662
f1_qrs_end  0.991706  0.002728  0.994720  0.988444  0.991345
f1_t_on     0.976889  0.005471  0.985296  0.971264  0.977334
f1_t_end    0.975626  0.007036  0.985552  0.965753  0.976131


: 